# Regressão em árvore

In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn.tree import DecisionTreeRegressor
import numpy as np

df = pd.read_csv("heavymachine_data.csv", sep=";")
output_colum = "Hours"
input_colums = ["Model","Aplication","Age"]
#retira coluna inutil
df.drop("Chassi", axis="columns", inplace=True)
#retira dados ruidosos
hours = df["Hours"]
std = hours.std()
mean = hours.mean()
df.drop(hours[hours > mean+std].index, inplace=True)

train_dataset = df.sample(frac=0.7,random_state=0)
test_dataset = df.drop(train_dataset.index)

df_dummys = pd.get_dummies(df, columns=["Aplication", "Model"])
train_ds_dummys = df_dummys.sample(frac=0.7,random_state=0)
test_ds_dummys = df_dummys.drop(train_dataset.index)

df.head()

,Hours,Model,Aplication,Age
0,461,111,71,11
1,277,90,39,10
2,905,87,35,10
3,437,87,72,10
4,311,90,35,10


# O modelo

## Não categórico

In [2]:
model = []
age = []
Aplication = []
y = []
x = []
for i,r in train_dataset.iterrows():    
    model.append(r["Model"])
    age.append(r["Age"])
    Aplication.append(r["Aplication"])
    x.append([r["Model"], r["Age"], r["Aplication"]])
    y.append(r["Hours"])
    # r.drop("Hours", inplace=True)
    # print(list(r))
    # break

# x = np.array(age).reshape(-1,1)
# x = np.array([model, age, Aplication])
# y = np.array(y)
regressor = DecisionTreeRegressor(max_depth=5)
model = regressor.fit(x, y)
r_sq = model.score(x, y)
print("Score: {}".format(r_sq))
errors = []
for i,r in test_dataset.iterrows():        
    y_true = pd.to_numeric(r["Hours"])
    r.drop("Hours", inplace=True)
    in_data = np.array(list(r)).reshape(-1,r.shape[0])
    # in_data = pd.DataFrame([{"Model": r["Model"], "Aplication": r["Aplication"], "Age":r["Age"]}])
    y_pred = model.predict(in_data)
    errors.append(abs(y_pred - y_true))
mae = np.mean(errors)
print("Mae: {}".format(mae))

Score: 0.6176838954423135
Mae: 4288.115162893786


## Categórico

In [6]:
y = []
x = []
for i,r in train_ds_dummys.iterrows():    
    y.append(r["Hours"])
    r.drop("Hours", inplace=True)
    x.append(list(r))

# x = np.array(age).reshape(-1,1)
# x = np.array([model, age, Aplication])
# y = np.array(y)
regressor = DecisionTreeRegressor(max_depth=5)
model_categorical = regressor.fit(x, y)
print("Score: {}".format(r_sq))
errors = []
for i,r in test_ds_dummys.iterrows():        
    y_true = pd.to_numeric(r["Hours"])
    r.drop("Hours", inplace=True)
    in_data = np.array(list(r)).reshape(-1,r.shape[0])    
    y_pred = model_categorical.predict(in_data)
    errors.append(abs(y_pred - y_true))
mae = np.mean(errors)
print("Mae: {}".format(mae))


Score: 0.6176838954423135
Mae: 1455.1826009857007
